In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

<b><font size="6" color="#009e84"> ch03. 한글 형태소 분석 / 시각화 / 유사도분석 / Quiz </font></b>

# ※ Quiz : 경주여행과 전주여행에 대해 최빈단어시각화와 유사도 분석

- (1) naver open API를 활용하여 네이버 지식인 '전주여행','경주여행'을 각각 500건씩 검색하여 백업(naver.csv)
    - replace(), re.sub()이용
    - 백업 파일 결과 : 검색어, no, title, link, description, total_text(title + ' ' + descricption)
- (2) 품사태깅 백업(naver_pos.csv)
    - query(검색어), no, token, pos(품사)
- (3) 명사만 추출 백업(naver_pos_nouns.csv)
    - query, token, pos
- (4) 빈도분석 백업(naver_pos_nouns_count.csv)
    - token, 경주빈도, 전주빈도, 빈도합
- (5) 빈도 시각화(워드클라우드, Text.plot)
    - 워드클라우드 이미지 저장
- (6) 단어간 유사도 분석(Word2Vec, 연관분석)

# 1. naver open API를 활용하여 검색어 500건 추출

- query, no, title, link, description, total_text(title + ' ' + descricption)